<a href="https://colab.research.google.com/github/praveenjune17/BERT_text_summarisation/blob/master/testset_eval/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import shutil
try:
  shutil.rmtree('/content/BERT_text_summarisation')
except:
  pass
!git clone https://github.com/praveenjune17/BERT_text_summarisation

Cloning into 'BERT_text_summarisation'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 631 (delta 9), reused 0 (delta 0), pack-reused 611
Receiving objects: 100% (631/631), 6.61 MiB | 11.32 MiB/s, done.
Resolving deltas: 100% (447/447), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')
!pip install bunch
!pip install rouge==0.3.2
!pip install bert_score 
!pip install --upgrade grpcio
!pip install tensorflow==2.1
!pip install tensorflow_addons

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
  Created wheel for bunch: filename=bunch-1.0.1-cp36-none-any.whl size=7076 sha256=4bc43faf6da17cf3531db2fc984d17ac10901c20616ec349b7b35a2fedcb22ba
  Stored in directory: /root/.cache/pip/wheels/56/0f/19/fbbf81e5764e6d8b74501c4357a88c14c94466ec777c03734c
Successfully built bunch
     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 481kB 11.2MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |██████████████

     |████████████████████████████████| 2.4MB 4.9MB/s 
  Found existing installation: grpcio 1.15.0
    Uninstalling grpcio-1.15.0:
      Successfully uninstalled grpcio-1.15.0


     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.8MB 34.9MB/s 
     |████████████████████████████████| 450kB 40.9MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


     |████████████████████████████████| 993kB 4.4MB/s 


In [5]:
%%shell
git clone https://github.com/praveenjune17/transformers.git
cd transformers
pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 17482, done.
remote: Total 17482 (delta 0), reused 0 (delta 0), pack-reused 17482
Receiving objects: 100% (17482/17482), 10.25 MiB | 14.41 MiB/s, done.
Resolving deltas: 100% (12980/12980), done.
Processing /content/transformers
  Created wheel for transformers: filename=transformers-2.3.0-cp36-none-any.whl size=458557 sha256=a0ab0c04696e66b0bf01edbb6dd5188a36c42ae5ab18bbe267c5e470f7d5b65e
  Stored in directory: /tmp/pip-ephem-wheel-cache-pw73yk7f/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 2.4.1
    Uninstalling transformers-2.4.1:
      Successfully uninstalled transformers-2.4.1


**Inference** **script**

Test set evaluation script

In [1]:
import sys
sys.path.insert(0, '/content/BERT_text_summarisation/scripts')
import tensorflow as tf
tf.random.set_seed(100)
import tensorflow_datasets as tfds
import numpy as np
import os
from create_tokenizer import tokenizer
from preprocess import map_batch_shuffle
from configuration import config
from input_path import file_path
from preprocess import infer_data_from_df
from metrics import convert_wordpiece_to_words
from rouge import Rouge
from bert_score import score as b_score
from decode_text import *
from tqdm import tqdm

UNK_ID = 100
CLS_ID = 101
SEP_ID = 102
MASK_ID = 103
h_parms.batch_size = 40
#ckpt = '59'
rouge_all = Rouge()
infer_template = '''Batch size <--- 30\
\nDraft_decoder_type <--- {}\nRefine_decoder_type <--- {}\nROUGE-f1  <--- {}\nBERT-f1   <--- {}\nbeam-size   <--- {}\nCheckpoint  <---{}'''


def restore_chkpt(checkpoint_path):
    ckpt = tf.train.Checkpoint(
                               model=model
                               )
    assert tf.train.latest_checkpoint(os.path.split(checkpoint_path)[0]), 'Incorrect checkpoint directory'
    ckpt.restore(checkpoint_path).expect_partial()
    print(f'{checkpoint_path} restored')
    
if config.use_tfds:
      examples, metadata = tfds.load(
                                     config.tfds_name, 
                                     with_info=True, 
                                     as_supervised=True, 
                                     data_dir='/content/drive/My Drive/Text_summarization/cnn_dataset',
                                     builder_kwargs={"version": "2.0.0"}
                                     )
      test_examples = examples['test']
      test_buffer_size = metadata.splits['test'].num_examples
      test_dataset = map_batch_shuffle(
                                       test_examples, 
                                       test_buffer_size, 
                                       split='test',
                                       batch_size=h_parms.batch_size
                                       )
      log.info('Test TF_dataset created')
      test_dataset = test_dataset.take(1)
else:
  test_dataset = infer_data_from_df()
for ckpt in range(73,74):
  ckpt = str(ckpt)
  try:
    restore_chkpt('/content/drive/My Drive/Text_summarization/BERT_text_summarisation/created_files/training_summarization_model_ckpts/cnn/best_checkpoints/ckpt-'+ckpt)
    
    ref_sents=[]
    hyp_sents=[]


    for (doc_id, (input_ids, _, _, target_ids, _, _)) in tqdm(enumerate(test_dataset, 1)):
      start_time = time.time()
      draft, refined_summary, att = predict_using_beam_search(
                                                              input_ids, 
                                                              beam_size=3, 
                                                              refine_decoder_sampling_type='greedy', 
    #                                                           k=k, 
    #                                                           p=p,
    #                                                           temperature=temperature
                                                              )
      for tar, ref_hyp in zip(target_ids, refined_summary):
        sum_ref = tokenizer.convert_ids_to_tokens([i for i in tf.squeeze(tar) if i not in [0, 101, 102]])
        sum_hyp = tokenizer.convert_ids_to_tokens([i for i in tf.squeeze(ref_hyp) if i not in [0, 101, 102]])
        sum_ref = convert_wordpiece_to_words(sum_ref)
        sum_hyp = convert_wordpiece_to_words(sum_hyp)
        #print('Original summary: {}'.format(sum_ref))
        #print('Predicted summary: {}'.format(sum_hyp))
        ref_sents.append(sum_ref)
        hyp_sents.append(sum_hyp)
      try:
        rouges = rouge_all.get_scores(ref_sents , hyp_sents)
        avg_rouge_f1 = np.mean([np.mean([rouge_scores['rouge-1']["f"], 
                                        rouge_scores['rouge-2']["f"], 
                                        rouge_scores['rouge-l']["f"]]) for rouge_scores in rouges])
        _, _, bert_f1 = b_score(ref_sents, hyp_sents, lang='en', model_type=config.pretrained_bert_model)
        avg_bert_f1 = np.mean(bert_f1.numpy())
      except:
        log.warning('predicted output is empty so setting rouge and bert score to 0')
        avg_rouge_f1 = 0
        avg_bert_f1 = 0
      print(infer_template.format('beam_search', 'greedy', avg_rouge_f1, avg_bert_f1, 3,ckpt))
      print(f'time to process document {doc_id} : {time.time()-start_time}')
      print(f'Calculating scores for {len(ref_sents)} golden summaries and {len(hyp_sents)} predicted summaries')
  except:
    print(f'{ckpt} checkpoint not found')
    continue

setting default value to last_recorded_value
INFO:tensorflow:Extracting pretrained word embeddings weights from BERT
INFO:tensorflow:Embedding matrix shape '(30522, 768)'
INFO:tensorflow:Loading Pre-trained BERT model for BERT SCORE calculation
INFO:tensorflow:Extracting pretrained word embeddings weights from BERT
INFO:tensorflow:Embedding matrix shape '(30522, 768)'


INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset cnn_dailymail (/content/drive/My Drive/Text_summarization/cnn_dataset/cnn_dailymail/plain_text/2.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /content/drive/My Drive/Text_summarization/cnn_dataset/cnn_dailymail/plain_text/2.0.0


INFO:tensorflow:Test TF_dataset created
66 checkpoint not found


0it [00:00, ?it/s]

/content/drive/My Drive/Text_summarization/BERT_text_summarisation/created_files/training_summarization_model_ckpts/cnn/best_checkpoints/ckpt-67 restored
INFO:tensorflow:Building: 'Draft beam search decoder'
INFO:tensorflow:Building: 'Refined greedy decoder'
draft_summary [40 73]
preds_refined_summary shape [40 72]


...

Batch size <--- 30
Draft_decoder_type <--- beam_search
Refine_decoder_type <--- greedy
ROUGE-f1  <--- 0.20650209426682267
BERT-f1   <--- 0.5106486678123474
beam-size   <--- 3
Checkpoint  <---71
time to process document 1 : 68.8572678565979
Calculating scores for 40 golden summaries and 40 predicted summaries



0it [00:00, ?it/s]

/content/drive/My Drive/Text_summarization/BERT_text_summarisation/created_files/training_summarization_model_ckpts/cnn/best_checkpoints/ckpt-72 restored
INFO:tensorflow:Building: 'Draft beam search decoder'
INFO:tensorflow:Building: 'Refined greedy decoder'
draft_summary [40 73]
preds_refined_summary shape [40 72]


1it [01:09, 69.51s/it]

Batch size <--- 30
Draft_decoder_type <--- beam_search
Refine_decoder_type <--- greedy
ROUGE-f1  <--- 0.211377583027235
BERT-f1   <--- 0.5072491765022278
beam-size   <--- 3
Checkpoint  <---72
time to process document 1 : 68.7435212135315
Calculating scores for 40 golden summaries and 40 predicted summaries


In [11]:
start=125000
ckpt_start=37
step = 5000
current_step = 230000
for i in range(125000,300000,step):
  
  ckpt_start+=1
  if i == current_step:
    print(ckpt_start)
    break
  else:
    continue
  

59


In [0]:
1it [02:02, 122.77s/it]Batch size <--- 30
Draft_decoder_type <--- beam_search
Refine_decoder_type <--- greedy
ROUGE-f1  <--- 0.20650209426682267
BERT-f1   <--- 0.5106486678123474
beam-size   <--- 3
time to process document 1 : 121.24681329727173
Calculating scores for 40 golden summaries and 40 predicted summaries
ckpt :- 71

In [0]:
Top-k's k   <--- 10
temperature   <--- 0.7
beam-size   <--- 5
p <-- 0.7, 0.8, 0.9, 1

In [0]:
0it [00:00, ?it/s]Batch size <--- 20
Draft_decoder_type <--- beam_search
Refine_decoder_type <--- topktopp
ROUGE-f1  <--- 0.1690104807952041
BERT-f1   <--- 0.48933693766593933
Nucleus's p   <--- 0.7
Topk's K  <--- 7
temperature   <--- 0.9
beam-size   <--- 4

In [0]:
# find best combo for top-k and top-p
# combined top-k and top-p

In [0]:
0it [00:00, ?it/s]Batch size <--- 20
Draft_decoder_type <--- beam_search
Refine_decoder_type <--- topk
ROUGE-f1  <--- 0.16558894018493717
BERT-f1   <--- 0.49077025055885315
Top-k's k   <--- 10
temperature   <--- 1
beam-size   <--- 3
time to process document 1 : 520.4894733428955

In [0]:
0it [00:00, ?it/s]Batch size <--- 20
Draft_decoder_type <--- beam_search
Refine_decoder_type <--- topk
ROUGE-f1  <--- 0.16919581924320998
BERT-f1   <--- 0.4894334375858307
Top-k's k   <--- 10
temperature   <--- 0.7
beam-size   <--- 5

In [0]:
# Nucleus sampling with batch
# replace k by p and run the hyper parameters script